In [14]:
import os
import pandas as pd
import statistics as stat
import numpy as np

# Helper functions

In [15]:
settings = {
    "baseALNS": {"folder": "../results/baseALNS", "parameters": "7200_8_11_0.1_1_0.005_1e-8_100_1e-4_2_0"},
    "(a)-ini": {"folder": "../results/exactMethods/a-ini", "parameters": "86400_8_ini_0"},
    "(b)-ini": {"folder": "../results/exactMethods/b-ini", "parameters": "86400_8_ini_0"},
    "(c)-ini": {"folder": "../results/exactMethods/c-ini", "parameters": "86400_8_ini_0"},
    "(a)": {"folder": "../results/exactMethods/a", "parameters": "86400_8_noIni_0"},
    "(b)": {"folder": "../results/exactMethods/b", "parameters": "86400_8_noIni_0"},
    "(c)": {"folder": "../results/exactMethods/c", "parameters": "86400_8_noIni_0"},
    "noHeuristicOperator": {"folder": "../results/impactOperators/removingHeuristicOperator", "parameters": "7200_8_01_0.1_1_0.005_1e-8_100_1e-4_2_0"},
    "noMILPOperator": {"folder": "../results/impactOperators/removingMILPOperator", "parameters": "7200_8_10_0.1_1_0.005_1e-8_100_1e-4_2_0"},
    "exponent0": {"folder": "../results/mechanismWeightUpdate/expo0", "parameters": "7200_8_11_0.1_1_0.005_1e-8_100_1e-4_0_0"},
    "exponent1": {"folder": "../results/mechanismWeightUpdate/expo1", "parameters": "7200_8_11_0.1_1_0.005_1e-8_100_1e-4_1_0"},
    "exponent1.5": {"folder": "../results/mechanismWeightUpdate/expo1.5", "parameters": "7200_8_11_0.1_1_0.005_1e-8_100_1e-4_1.5_0"},
    "exponent2.5": {"folder": "../results/mechanismWeightUpdate/expo2.5", "parameters": "7200_8_11_0.1_1_0.005_1e-8_100_1e-4_2.5_0"},
    "exponent3": {"folder": "../results/mechanismWeightUpdate/expo3", "parameters": "7200_8_11_0.1_1_0.005_1e-8_100_1e-4_3_0"},

}

instances = [
 "S1", "S2", "S3",
 "M1", "M2", "M3",
 "L1", "L2", "L3"
 ]

In [16]:
def get_values(instance, settings, name):
    setting = settings[name]
    for stat in ("values", "mean", "best", "CV"):
        setting[stat + "_" + instance] = []

    inputFiles = os.listdir(setting["folder"])
    for inputFile in inputFiles:
        splited = inputFile.split("_")
        instanceRead = splited[1]
        if instanceRead == instance:
            with open(os.path.join(setting["folder"], inputFile), 'r') as file:
                value = float(file.read().strip())
                setting["values_" + instance].append(value)

    setting["mean_" + instance] = np.mean(setting["values_" + instance])
    setting["best_" + instance] = min(setting["values_" + instance])
    if len(setting["values_" + instance]) > 1:
        setting["CV_" + instance] = np.std(setting["values_" + instance]) / setting["mean_" + instance]

    return setting

# Comparison to Gurobi and the exact solution methods in Lee et al. (2022) (Section 5.2, Table 2)

In [17]:
index = pd.Index(["ALNS^avg_2h", "Gap_ALNS^(a)-ini", "Gap_ALNS^(b)-ini", "Gap_ALNS^(c)-ini", "Gap_ALNS^(a)", "Gap_ALNS^(b)", "Gap_ALNS^(c)", "ALNS^CV_2h"])
df = pd.DataFrame(index = index, columns = instances)
for instance in instances:
    baseALNS = get_values(instance, settings, "baseALNS")
    df.loc["ALNS^avg_2h", instance] = int(round(baseALNS["mean_" + instance], 0))
    df.loc["ALNS^CV_2h", instance] = str(round(100 * baseALNS["CV_" + instance], 2)) + " %"

    for name in ["(a)-ini", "(b)-ini", "(c)-ini"]: # ini
        exactMethod = get_values(instance, settings, name)
        gap = ((baseALNS["mean_" + instance] - exactMethod["mean_" + instance]) / exactMethod["mean_" + instance]) * 100
        df.loc["Gap_ALNS^" + name, instance] = str(round(gap, 1)) + " %"

    for name in ["(a)", "(b)", "(c)"]: # no ini
        exactMethod = get_values(instance, settings, name)
        gap = ((baseALNS["mean_" + instance] - exactMethod["best_" + instance]) / exactMethod["best_" + instance]) * 100
        if np.isnan(gap):
            df.loc["Gap_ALNS^" + name, instance] = "N/A"
        else:
            df.loc["Gap_ALNS^" + name, instance] = str(round(gap, 1)) + " %"
df

c:\Users\rpaga\Anaconda3\envs\Paper1\lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\rpaga\AppData\Local\Temp\ipykernel_24092\688793411.py:15: RuntimeWarning: invalid value encountered in scalar divide
  gap = ((baseALNS["mean_" + instance] - exactMethod["best_" + instance]) / exactMethod["best_" + instance]) * 100


,S1,S2,S3,M1,M2,M3,L1,L2,L3
ALNS^avg_2h,2217,1894,2427,4584,3875,4936,9842,8474,10784
Gap_ALNS^(a)-ini,-1.9 %,-2.2 %,-3.2 %,-5.3 %,-5.9 %,-6.5 %,-17.9 %,-21.7 %,-22.4 %
Gap_ALNS^(b)-ini,-1.8 %,-2.8 %,-2.6 %,-22.2 %,-12.5 %,-11.7 %,-32.6 %,-35.5 %,-29.9 %
Gap_ALNS^(c)-ini,-2.0 %,-3.6 %,-3.0 %,-37.1 %,-44.7 %,-40.6 %,-37.0 %,-38.3 %,-36.2 %
Gap_ALNS^(a),-4.3 %,-4.2 %,-3.6 %,-12.3 %,-16.8 %,-6.3 %,N/A,N/A,N/A
Gap_ALNS^(b),-4.1 %,-4.3 %,-7.1 %,N/A,N/A,N/A,N/A,N/A,N/A
Gap_ALNS^(c),-4.2 %,-4.3 %,-2.0 %,N/A,N/A,-34.1 %,N/A,N/A,N/A
ALNS^CV_2h,0.56 %,0.39 %,0.43 %,0.83 %,0.66 %,1.01 %,0.7 %,0.89 %,0.41 %


# Impact of each destroy-operator (Section 5.5, Table 4)

In [23]:
index = pd.Index(["Gap_noHeuristicOperator", "Gap_noMILPOperator"])
df = pd.DataFrame(index = index, columns = instances)
for instance in instances:
    baseALNS = get_values(instance, settings, "baseALNS")
    for name in ["noHeuristicOperator", "noMILPOperator"]:
        other = get_values(instance, settings, name)
        gap = ((other["mean_" + instance] - baseALNS["mean_" + instance]) / baseALNS["mean_" + instance]) * 100
        df.loc["Gap_" + name, instance] = str(round(gap, 1)) + " %"
df

,S1,S2,S3,M1,M2,M3,L1,L2,L3
Gap_noHeuristicOperator,0.6 %,0.2 %,0.7 %,0.8 %,0.6 %,1.0 %,2.0 %,1.6 %,1.9 %
Gap_noMILPOperator,5.5 %,6.6 %,4.5 %,7.0 %,8.0 %,8.1 %,3.5 %,2.9 %,4.1 %


# Impact of the mechanism updating the weights (Section 5.6, Table 5)

In [19]:
index = pd.Index(["Gap_exponent0", "Gap_exponent1", "Gap_exponent1.5", "Gap_exponent2.5", "Gap_exponent3"])
df = pd.DataFrame(index = index, columns = instances)
for instance in instances:
    baseALNS = get_values(instance, settings, "baseALNS")
    for name in ["exponent0", "exponent1", "exponent1.5", "exponent2.5", "exponent3"]:
        other = get_values(instance, settings, name)
        gap = ((other["mean_" + instance] - baseALNS["mean_" + instance]) / baseALNS["mean_" + instance]) * 100
        df.loc["Gap_" + name, instance] = str(round(gap, 1)) + " %"
df

,S1,S2,S3,M1,M2,M3,L1,L2,L3
Gap_exponent0,0.5 %,0.6 %,0.3 %,0.8 %,0.8 %,0.6 %,1.8 %,0.7 %,1.4 %
Gap_exponent1,0.2 %,0.4 %,-0.0 %,0.6 %,0.7 %,0.4 %,1.6 %,1.2 %,2.0 %
Gap_exponent1.5,-0.1 %,0.4 %,0.4 %,-0.0 %,-0.3 %,-0.3 %,0.7 %,-0.6 %,0.4 %
Gap_exponent2.5,1.1 %,1.5 %,1.3 %,1.7 %,1.2 %,1.4 %,-0.1 %,-0.3 %,0.5 %
Gap_exponent3,5.9 %,6.5 %,4.6 %,6.1 %,6.2 %,6.2 %,2.2 %,2.7 %,3.1 %
